In [1]:
%%capture
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

2024-03-05 23:09:32.226800: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-05 23:09:32.322812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 23:09:32.322958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 23:09:32.327379: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 23:09:32.350064: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from PIL import Image
image = Image.open("cat.jpg")
inputs = processor(
    text=["a photo of a cat", "a photo of a dog"],
    images=image,
    return_tensors="pt",
    padding=True,
)

In [3]:
from video_processor import VideoProcessor

In [5]:
from transformers import AutoImageProcessor, VideoMAEModel
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")

In [7]:
vp = VideoProcessor(
    clip_model.text_model,
    model,
    processor.tokenizer,
    image_processor,
    device="cpu",
)
video,audio = vp.extract_frames_and_audio(
    "/DATA/sarmistha_2221cs21/basha/VideoMAE/dataset_224x224/_gPgueccVzw.mp4"
)

Extracting frame chunks


100%|██████████| 10908/10908 [04:24<00:00, 41.28it/s]
/DATA/sarmistha_2221cs21/basha/VideoMAE/Design/video_processor.py:139: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, _ = librosa.load(video_path, sr=16000)
/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Extracting audio chunks


100%|██████████| 188/188 [00:00<00:00, 211742.52it/s]


In [8]:
import pickle
with open("frames_audio__gPgueccVzw.pkl",'wb') as f:
    pickle.dump([video,audio],f)

In [14]:
encoding = vp.vision_encoder(video)

Encoding frames


/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(value)


In [18]:
import os
os.getcwd()
video_dir = '/home/basha_2211ai03/complaint_detection/videos/dataset'
videos = os.listdir(video_dir)

In [19]:
videos[0]

'wNgOKJpk7K0.mp4'

In [22]:
import pickle
try:
    with open("data.pkl",'rb') as f:
        encoded_data = pickle.load(f)
except:
    encoded_data = {}

In [23]:
len(encoded_data)

5

In [25]:

import torch
import pickle
from tqdm.notebook import tqdm
with torch.inference_mode():
    for video in tqdm(videos):
        if video in encoded_data:
            continue
        out = vp(os.path.join(video_dir,video))
        encoded_data[video] = out
        with open("data.pkl",'wb') as f:
            pickle.dump(encoded_data,f)

  0%|          | 0/408 [00:00<?, ?it/s]

NoBackendError: 

In [6]:
import pickle
with open("data.pkl",'rb') as f:
    out = pickle.load(f)

In [9]:
vp = vp.eval()

In [9]:
import av
import numpy as np

from transformers import AutoImageProcessor, VideoMAEModel
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")

# prepare video for the model
inputs = image_processor(list(video), return_tensors="pt")

# forward pass
outputs = model(**inputs)

eating_spaghetti.mp4:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

In [15]:
outputs.last_hidden_state.mean(1).squeeze().shape

torch.Size([768])

In [16]:
clip_model

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [1]:
import pickle
with open("data.pkl",'rb') as f:
    data = pickle.load(f)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
out = vp.image_model(**processor.image_processor([image,image], return_tensors="pt").to("cuda"))

In [ ]:
out.pooler_output.mean(axis=0).shape

In [ ]:
processor.tokenizer('basha')

In [ ]:

out = clip_model.text_model(
    input_ids = inputs['input_ids'],
    attention_mask = inputs['attention_mask']
)

In [ ]:
out.last_hidden_state.shape

In [ ]:
out.pooler_output

In [ ]:
out.last_hidden_state[0]